In [8]:
# 連接 Elasticsearch

In [9]:
from datetime import datetime
from elasticsearch import Elasticsearch
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

In [10]:
es_ip = "http://127.0.0.1:9200"

In [11]:
es = Elasticsearch(es_ip)

In [12]:
# 查詢資料(讀取單一筆資料)

In [13]:
res = es.get(index="winlogbeat", id="IM3k130BMs_nTerjgcWJ")
print(res['_source'])

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B250>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B250>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。)

In [ ]:
# 查詢資料(取得前100筆資料)

In [ ]:
query_str = '{"query":{"range":{"@timestamp":{"gte":"2021-12-20", "lte":"2021-12-20"}}}}'
res = es.search(index="winlogbeat", body=query_str, from_=0, size=100)

In [ ]:
# 取得筆數

In [ ]:
total_hits = res["hits"]["total"]
print("total : " + str(total_hits))

In [14]:
# 顯示 100 筆資料

In [15]:
print(res["hits"]["hits"])

NameError: name 'res' is not defined

In [16]:
# Show Event Count by Datetime

In [17]:
query_str = {"size": 0,"aggs": {"result": {"date_histogram": {"field": "@timestamp","interval": "1d","format": "MM-dd"}}}}
res = es.search(index="winlogbeat", body=query_str)
result = res["aggregations"]["result"]["buckets"]
#print(result)

C:\Users\enya\AppData\Local\Temp/ipykernel_6128/3439329284.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="winlogbeat", body=query_str)


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B880>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B880>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。)

In [ ]:
event_pd = pd.DataFrame(result, columns=["key_as_string", "doc_count"])
#print(event_pd)
event_pd.plot(x="key_as_string", y="doc_count", kind="bar");
plt.xlabel('Date')
plt.ylabel('Log Count')
plt.title('Event Logs')

In [ ]:
# Show Event Count by EventID

In [ ]:
query_str = {"size": 0,"aggregations": {"result": {"terms": {"field": "winlog.event_id.keyword","order": [{"_count": "desc"}]}}}}
res = es.search(index="winlogbeat", body=query_str)
result = res["aggregations"]["result"]["buckets"]
#print(result)

In [ ]:
event_pd = pd.DataFrame(result, columns=["key", "doc_count"])
#print(event_pd)
event_pd.plot(x="key", y="doc_count", kind="bar");
plt.xlabel('Event ID')
plt.ylabel('Log Count')
plt.title('Event Logs')

In [ ]:
# Show Event Count by DestinationIp

In [18]:
query_str = {"size": 0,"aggregations": {"result": { "terms": {"field": "winlog.event_data.DestinationIp.keyword","order": [{"_count": "desc"}]}}}}
res = es.search(index="winlogbeat", body=query_str)
result = res["aggregations"]["result"]["buckets"]
#print(result)

C:\Users\enya\AppData\Local\Temp/ipykernel_6128/3936957868.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="winlogbeat", body=query_str)


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B2B0>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000024E0D30B2B0>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。)

In [ ]:
event_pd = pd.DataFrame(result, columns=["key", "doc_count"])
#print(event_pd)
event_pd.plot(x="key", y="doc_count", kind="bar");
plt.xlabel('Event ID')
plt.ylabel('Log Count')
plt.title('Event Logs')

In [ ]:
# IP查詢

In [ ]:
request_headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) ' \
    'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'
}
for ip in event_pd["key"]:
    print(ip)
    req = requests.get("https://ipapi.co/"+ip+"/json", headers=request_headers)
    print(req.json())
    time.sleep(5)